### 1. 광주 광역시 건축 데이터 데이터 수집 
<br>

방법 : 전국 시/도 중 광주 광역시의 각 동마다 건물 데이터를 수집  
<br>

위치 : 건축데이터 민간개방 시스템 (https://open.eais.go.kr/main/main.do)
<br> 
상세위치 : 건축데이터 민간개방 시스템 - 개방서비스 - 유형별 건축데이터 - 건축물대장 - 표제부

### 2. 광주 광역시 건축 데이터 전처리
<br> 

※ 수집한 건물 수 <br> 
　[ 광산구 : 43266 ] [ 동구 : 27711 ]  [ 서구 : 23854 ]  [ 남구 : 38585 ] [ 북구 : 57515 ]  
<br> 
  
※ 데이터 컬럼 <br> 
　대지위치, 도로명대지위치, 건물명, 특수지명, 블록, 로트, 번지, 동명칭,   
　대지면적(m^2), 건축면적(m^2), 건폐율(%), 연면적(m^2), 용적률산정연면적(m^2),   
　용적률(%), 주용도코드명, 기타용도, 높이(m), 지상층수, 지하층수, 허가일, 착공일,   
　사용승인일, 허가번호년, 생성일자, 내진설계적용여부, 내진능력  
 
 <br>

### 수행할 프로세스
#### process 1. 위/경도 수집  
#### process 2. 중복된 주소 제거  


### process2. 중복된 주소 제거
<br>

1. 기존 리스트 a에서 중복을 제거한 새로운 주소 리스트 b생성
2. a 리스트에서 중복된 주소의 개수를 count한 리스트 c생성
3. a 리스트에서 b 리스트와 중복되는 인덱스 리스트 d 생성  
4. 제거될 주소 중에서 가장 큰 값을 남김  
<br> 

리스트 생성 예)   
a = ['a', 'a', 'a', 'b', 'b', 'c', 'd', 'd']  
b = ['a', 'b', 'c', 'd']  
c = [3, 2, 1, 2]  
d = [3, 5, 6, 8]  


In [ ]:
import pandas as pd
import time

start = time.time()
  
'''
process1-1
'''
print('*** process 1 ***')

processing=pd.read_csv('./건축물대장표제부-광주광역시.csv', low_memory=False).fillna('') # 데이터 불러옴, (리스트 a)

print('*** "time :', time.time() - start, 'process 1 fin ***')


'''
process1-2
'''
print('*** process 2 ***')

location=[] # 중복되지 않은 주소 (리스트 b)

for v in processing['대지위치']:
    if v not in location:
        location.append(v)
    
print('*** "time :', time.time() - start, 'process 2 fin ***')


'''
process1-3
'''
print('*** process 3 ***')

w_count= {} # a 리스트의 중복된 주소 카운트 (리스트 c)
n_count= {} # a 리스트에서 b 리스트와 중복되는 주소 카운트 (리스트 d)

for i in processing['대지위치']:
    try: 
        w_count[i]+= 1
        n_count[i]+= 1
    except: 
        w_count[i]=1
        n_count[i]=1

for x in range(1, len(location)):
    n_count[location[x]]=n_count[location[x]]+n_count[location[x-1]]

print('*** "time :', time.time() - start, 'process 3 fin ***')


'''
process1-4
'''
print('*** process 4 ***')

usetmp=[]
seismictmp=[]

for x in range(0, len(processing)): 
    if processing['사용승인일'][x]=='': # 빈 칸들 1960년으로 채움
        usetmp.append(1960)
        
    elif processing['사용승인일'][x]!=0:  # 날짜에서 월/일 제거
        _tmp=str(processing['사용승인일'][x])[:4]        
        usetmp.append(int(_tmp))

    if processing['내진설계적용여부'][x] == '':
        seismictmp.append(int(0))
    elif processing['내진설계적용여부'][x] != '':
        seismictmp.append(int(processing['내진설계적용여부'][x]))

processing['사용승인일']      = usetmp
processing['내진설계적용여부'] = seismictmp

print('*** "time :', time.time() - start, 'process 4 fin ***')


'''
process1-5
'''    
print('*** process 5 ***')

Building_lat=[]          # 건물위도
Building_lng=[]          # 건물경도
Building_structure=[]    # 건물구조
Building_area=[]         # 건물면적
Building_height=[]       # 건물높이
Building_Ground=[]       # 지상층수
Building_UnderGround=[]  # 지하층수
Building_date=[]         # 사용승인일
Building_Seismic=[]      # 건물내진설계

for x in range(0, len(location)):
    try:            
        if w_count[location[x]] == 1:
            lat_tmp       =processing['위도'][n_count[location[x-1]]]
            lng_tmp       =processing['경도'][n_count[location[x-1]]]
            structure_tmp =processing['건물구조'][n_count[location[x-1]]]
            area_tmp      =processing['건축면적(㎡)'][n_count[location[x-1]]]
            height_tmp    =processing['높이(m)'][n_count[location[x-1]]]
            ground_tmp    =processing['지상층수'][n_count[location[x-1]]]
            under_tmp     =processing['지하층수'][n_count[location[x-1]]]
            date_tmp      =processing['사용승인일'][n_count[location[x-1]]]
            seismic_tmp   =processing['내진설계적용여부'][n_count[location[x-1]]]
                        
            Building_lat.append(lat_tmp)
            Building_lng.append(lng_tmp)            
            Building_structure.append(structure_tmp)
            Building_area.append(area_tmp)
            Building_height.append(height_tmp)
            Building_Ground.append(ground_tmp)
            Building_UnderGround.append(under_tmp)
            Building_date.append(date_tmp)
            Building_Seismic.append(seismic_tmp)
            
        if w_count[location[x]] > 1:
            if x == 0:
                lat_tmp       =processing['위도'][0]
                lng_tmp       =processing['경도'][0]
                structure_tmp =processing['건물구조'][0]
                area_tmp      =processing['건축면적(㎡)'][0]
                height_tmp    =processing['높이(m)'][0]
                ground_tmp    =processing['지상층수'][0]
                under_tmp     =processing['지하층수'][0]
                date_tmp      =processing['사용승인일'][0]
                seismic_tmp   =processing['내진설계적용여부'][0]
                
                _count = n_count[location[0]]
                for y in range(0, n_count[location[x]]):
                    if processing['건축면적(㎡)'][y] > area_tmp:
                        area_tmp=processing['건축면적(㎡)'][y]
                        _count=y
                        structure_tmp=processing['건물구조'][_count]
                    if processing['높이(m)'][y] > height_tmp:
                        height_tmp=processing['높이(m)'][y]
                    if processing['지상층수'][y] > ground_tmp:
                        ground_tmp=processing['지상층수'][y]
                    if processing['지하층수'][y] > under_tmp:
                        under_tmp=processing['지하층수'][y]
                    if processing['사용승인일'][y] > date_tmp:
                        date_tmp=processing['사용승인일'][y]
                    if processing['내진설계적용여부'][y] > seismic_tmp:
                        seismic_tmp=processing['내진설계적용여부'][y]
                
                Building_lat.append(lat_tmp)
                Building_lng.append(lng_tmp)            
                Building_structure.append(structure_tmp)
                Building_area.append(area_tmp)
                Building_height.append(height_tmp)
                Building_Ground.append(ground_tmp)
                Building_UnderGround.append(under_tmp)
                Building_date.append(date_tmp)
                Building_Seismic.append(seismic_tmp)
                continue

            lat_tmp       =processing['위도'][n_count[location[x-1]]]
            lng_tmp       =processing['경도'][n_count[location[x-1]]]
            structure_tmp =processing['건물구조'][n_count[location[x-1]]]
            area_tmp      =processing['건축면적(㎡)'][n_count[location[x-1]]]
            height_tmp    =processing['높이(m)'][n_count[location[x-1]]]
            ground_tmp    =processing['지상층수'][n_count[location[x-1]]]
            under_tmp     =processing['지하층수'][n_count[location[x-1]]]
            date_tmp      =processing['사용승인일'][n_count[location[x-1]]]
            seismic_tmp   =processing['내진설계적용여부'][n_count[location[x-1]]]
            
            _count = n_count[location[x-1]]
            for y in range(n_count[location[x-1]], n_count[location[x]]):
                if processing['건축면적(㎡)'][y] > area_tmp:
                    area_tmp=processing['건축면적(㎡)'][y]
                    _count=y
                    structure_tmp=processing['건물구조'][_count]
                if processing['높이(m)'][y] > height_tmp:
                    height_tmp=processing['높이(m)'][y]
                if processing['지상층수'][y] > ground_tmp:
                    ground_tmp=processing['지상층수'][y]
                if processing['지하층수'][y] > under_tmp:
                    under_tmp=processing['지하층수'][y]
                if processing['사용승인일'][y] > date_tmp:
                    date_tmp=processing['사용승인일'][y]
                if processing['내진설계적용여부'][y] > seismic_tmp:
                    seismic_tmp=processing['내진설계적용여부'][y]
                                
            Building_lat.append(lat_tmp)
            Building_lng.append(lng_tmp)            
            Building_structure.append(structure_tmp)
            Building_area.append(area_tmp)
            Building_height.append(height_tmp)
            Building_Ground.append(ground_tmp)
            Building_UnderGround.append(under_tmp)
            Building_date.append(date_tmp)
            Building_Seismic.append(seismic_tmp)
            
    except IndexError:
        pass
    
print('*** "time :', time.time() - start, 'process 5 fin ***')


'''
process1-6
'''    
print('*** process 6 ***')

area     =pd.DataFrame(Building_area,        columns=['area'])
height   =pd.DataFrame(Building_height,      columns=['height'])
ground   =pd.DataFrame(Building_Ground,      columns=['ground'])
uground  =pd.DataFrame(Building_UnderGround, columns=['underground'])
seismic  =pd.DataFrame(Building_Seismic,     columns=['seismic'])
location =pd.DataFrame(location,             columns=['location'])
structure=pd.DataFrame(Building_structure,   columns=['structure'])
lat      =pd.DataFrame(Building_lat,         columns=['lat'])
lon      =pd.DataFrame(Building_lng,         columns=['lon'])
date     =pd.DataFrame(Building_date,        columns=['date'])

final=pd.concat([location, lat, lon, structure, area, height, ground, uground, date, seismic], axis=1)
final.to_excel('./DataProcessing_광주광역시건축물.xlsx', index=False)

print('*** "time :', time.time() - start, 'process 6 fin ***')
